In [1]:
import cvxpy as cp
import numpy as np

# DGP Approach 1 - Beta as variable

In [2]:
k_cats = cp.Variable(6, pos=True) # (1/sec)
K_ms = cp.Variable(6, pos=True) # K_M (mol/L)
BETA   = cp.Variable(1, pos=True) # unitless
# k_cats_diff_slack = cp.Variable(1, pos=True) # (1/sec)
# K_m_diff_slack = cp.Variable(1, pos=True) # (mol/L)
S = cp.Variable(6, pos=True) # (mol/L)

P_L = 0.00014218052336201800 # (mol/L)
k_cat_exp = 0.071 # (1/sec)
K_ms_exp = 3.7E-6 # (mol/L)
#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER
S_protein_concs = np.array([4.46510054478746E-07, 0.00027087422911447700, 0.0006261898295372690, 0.0006151283915983840, 8.64621184397196E-06, 0.0006831411710823680]) # (mol/L)
S_std_dev = np.array([6.48605511482799E-07, 0.00017348313510493000, 0.00014728328233160000, 0.0002649777812708530, 2.04927424163342E-05, 0.00013085185087382300]) # (mol/L)
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275])*60 # (sec)

# construct constraints
constraints = [BETA >= cp.sum(cp.multiply(cp.inv_pos(K_ms),S)) + 1,
               S <= S_protein_concs + S_std_dev]
for idx, (concs, dev) in enumerate(zip(S_protein_concs, S_std_dev)):
    if concs - dev > 0:
        constraints += [S[idx] >= concs - dev]
    else:
        constraints += [S[idx] >= 0.01 * concs]
# if S_protein_concs[0] - S_std_dev[0] > 0:
#     constraints += [S >= S_protein_concs - S_std_dev]
# else:
#     constraints += [S >= 0.01 * S_protein_concs]
# constraints += [S >= 0.5 * S_protein_concs]
constraints += [cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * BETA)) >= np.log(2)/t_half]

# construct objective
rel_diff_k_cats = cp.maximum(k_cats / k_cat_exp, k_cat_exp / k_cats) 
rel_diff_K_ms = cp.maximum(K_ms / K_ms_exp, K_ms_exp / K_ms)

obj_k_cats = cp.sum(rel_diff_k_cats) # minimize the sum of relative difference k_cats
obj_K_ms = cp.sum(rel_diff_K_ms) # minimize the sum of relative difference K_ms
obj_deg_rate = cp.sum(cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * BETA))) # minimize to achieve ..
objectives = [obj_k_cats, obj_K_ms, obj_deg_rate]

# construct problem
problem = cp.Problem(cp.Minimize(obj_k_cats + obj_K_ms + obj_deg_rate + BETA), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

86.10093301478447

In [3]:
print('Simulated value of beta:', BETA.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: [39.5443234]
Simulated value of k_cats: [0.071 0.071 0.071 0.071 0.071 0.071]
Simulated value of K_ms: [3.70000004e-06 1.89666166e-05 4.20152305e-05 3.59292970e-05
 3.70000005e-06 4.51182510e-05]
Simulated value of S: [4.46527853e-09 9.73910950e-05 4.78906550e-04 3.50150613e-04
 8.64622635e-08 5.52289323e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: 0.16996764807725184


In [4]:
S_protein_concs

array([4.46510054e-07, 2.70874229e-04, 6.26189830e-04, 6.15128392e-04,
       8.64621184e-06, 6.83141171e-04])

In [5]:
obj_deg_rate.value

0.17028662917139

In [6]:
np.sum(np.log(2)/t_half)

0.0003189810941381753

In [7]:
BETA.value, np.sum(np.multiply(1/(K_ms.value),S.value)) + 1

(array([39.5443234]), 39.54432292639752)

In [8]:
rel_diff_k_cats.value

array([1., 1., 1., 1., 1., 1.])

# DGP Appraoch 2 - Beta as parameter

In [9]:
k_cats = cp.Variable(6, pos=True) # (1/sec)
K_ms = cp.Variable(6, pos=True) # K_M (mol/L)
S = cp.Variable(6, pos=True) # (mol/L)

P_L = 0.00014218052336201800 # (mol/L)
k_cat_exp = 0.071 # (1/sec)
K_ms_exp = 3.7E-6 # (mol/L)
#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER
S_protein_concs = np.array([4.46510054478746E-07, 0.00027087422911447700, 0.0006261898295372690, 0.0006151283915983840, 8.64621184397196E-06, 0.0006831411710823680]) # (mol/L)
S_std_dev = np.array([6.48605511482799E-07, 0.00017348313510493000, 0.00014728328233160000, 0.0002649777812708530, 2.04927424163342E-05, 0.00013085185087382300]) # (mol/L)
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275])*60 # (sec)

# construct constraints
# 1. Define beta as a parameter
beta = cp.sum(cp.multiply(cp.inv_pos(K_ms),S)) + 1

# 2. Define range of feasible substrate concentrations
constraints = [S <= S_protein_concs + S_std_dev]
for idx, (concs, dev) in enumerate(zip(S_protein_concs, S_std_dev)):
    if concs - dev > 0:
        constraints += [S[idx] >= concs - dev]
    else:
        constraints += [S[idx] >= 0.01 * concs]

# 3. Define relationship between k_cats, K_ms, and half life that we try to minimize the difference of 
constraints += [cp.multiply(P_L*k_cats, cp.inv_pos(K_ms * beta)) >= np.log(2)/t_half]

constraints += [k_cats >= k_cat_exp] # add constraint on k_cats
# construct objective
# 1. Define relative error between estimation and experimental values
rel_diff_k_cats = cp.maximum(k_cats / k_cat_exp, k_cat_exp / k_cats) 
rel_diff_K_ms = cp.maximum(K_ms / K_ms_exp, K_ms_exp / K_ms)
obj_k_cats = cp.sum(rel_diff_k_cats) # minimize the sum of relative difference k_cats
obj_K_ms = cp.sum(rel_diff_K_ms) # minimize the sum of relative difference K_ms

# 2. Define objective to minimize the degradation rate
k_degs = [cp.quad_over_lin(P_L*k_cats[i], cp.inv_pos(K_ms[i] * beta)) for i in range(6)] # wrong equation
obj_deg_rate = cp.sum(k_degs) # minimize to achieve degradation rate
objectives = [obj_k_cats, obj_K_ms, obj_deg_rate]

# construct problem
problem = cp.Problem(cp.Minimize(obj_k_cats + obj_K_ms + obj_deg_rate), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

12.000000014182937

In [10]:
print('Simulated value of beta:', beta.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: 497.61419370801417
Simulated value of k_cats: [0.071 0.071 0.071 0.071 0.071 0.071]
Simulated value of K_ms: [3.7e-06 3.7e-06 3.7e-06 3.7e-06 3.7e-06 3.7e-06]
Simulated value of S: [7.36131357e-07 1.83295263e-04 5.59012611e-04 4.49440584e-04
 1.62156143e-05 6.28772313e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: -0.00031898109301242353


In [11]:
# Solve for problem without obj_k_cats and obj_K_ms
problem = cp.Problem(cp.Minimize(obj_deg_rate), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

5.629708783161636e-13

In [12]:
print('Simulated value of beta:', beta.value)
print('Simulated value of k_cats:', k_cats.value) 
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Simulated value of beta: 250801922.3311549
Simulated value of k_cats: [0.07100005 0.071      0.071      0.071      0.07100001 0.071     ]
Simulated value of K_ms: [1.98004046e-14 2.92458290e-12 6.48556638e-12 5.54538677e-12
 8.71391375e-14 6.96466635e-12]
Simulated value of S: [4.46510645e-09 9.73910954e-05 4.78906551e-04 3.50150613e-04
 8.64621455e-08 5.52289324e-04]
Simulated difference between obj_deg_rate and ln(2)/half_life: -0.0003189810935752044


# DGP Appraoch 3 - Same kcats for all, Beta as parameter

In [25]:
K_ms = cp.Variable(6, pos=True) # K_M (mol/L)
S = cp.Variable(6, pos=True) # (mol/L)

P_L = 0.00014218052336201800 # (mol/L)
k_cat = 0.071 # (1/sec)
K_ms_exp = 3.7E-6 # (mol/L)

#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER
S_protein_concs = np.array([4.46510054478746E-07, 0.00027087422911447700, 0.0006261898295372690, 0.0006151283915983840, 8.64621184397196E-06, 0.0006831411710823680]) # (mol/L)
S_std_dev = np.array([6.48605511482799E-07, 0.00017348313510493000, 0.00014728328233160000, 0.0002649777812708530, 2.04927424163342E-05, 0.00013085185087382300]) # (mol/L)
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275])*60 # (sec)

# ---Construct constraints---
# 1. Define beta as a parameter
beta = cp.sum(cp.multiply(cp.inv_pos(K_ms),S)) + 1

# 2. Define the rate of active degradation k_active
k_active = cp.multiply(P_L*k_cat, cp.inv_pos(K_ms * beta))

# 3. Define range of feasible substrate concentrations
constraints = [S <= S_protein_concs + S_std_dev]
for idx, (concs, dev) in enumerate(zip(S_protein_concs, S_std_dev)):
    if concs - dev > 0:
        constraints += [S[idx] >= concs - dev]
    else:
        constraints += [S[idx] >= 0.01 * concs]
        
# 4. Define relationship between k_cats, K_ms, and half life that we try to minimize the difference of
constraints += [k_active >= np.log(2)/t_half]

# ---Construct objective---
# 1. Define relative error between estimation and experimental values
rel_diff_K_ms = cp.maximum(K_ms / K_ms_exp, K_ms_exp / K_ms)
obj_K_ms = cp.sum(rel_diff_K_ms) # minimize the sum of relative difference K_ms

# 2. Define objective to minimize the degradation rate
k_degs = [cp.quad_over_lin(P_L*k_cat, cp.inv_pos(K_ms[i] * beta)) for i in range(6)]
obj_deg_rate = cp.sum(k_degs) # minimize to achieve degradation rate
objectives = [obj_K_ms, obj_deg_rate]

# construct problem
problem = cp.Problem(cp.Minimize(obj_K_ms + obj_deg_rate + beta), constraints)
problem.solve(gp=True, solver=cp.CLARABEL)

79.93052536362677

In [26]:
print('Value used for constant k_cat:', k_cat) 
print('Simulated value of beta:', beta.value)
print('Simulated value of K_ms:', K_ms.value) # show 2 decimal places
print('Simulated value of S:', S.value)
print('Simulated value of active degradation rates k_active:', k_active.value)
print('Simulated difference between obj_deg_rate and ln(2)/half_life:', obj_deg_rate.value - np.sum(np.log(2)/t_half))

Value used for constant k_cat: 0.071
Simulated value of beta: 39.47762411863032
Simulated value of K_ms: [3.70000001e-06 1.89826328e-05 4.20946414e-05 3.59933906e-05
 3.70000001e-06 4.52050699e-05]
Simulated value of S: [4.46519420e-09 9.73910946e-05 4.78906549e-04 3.50150612e-04
 8.64622165e-08 5.52289322e-04]
Simulated value of active degradation rates k_active: [0.06911077 0.01347073 0.00607464 0.00710436 0.06911077 0.00565666]
Simulated difference between obj_deg_rate and ln(2)/half_life: -0.00031898109353603273


# DGP Appraoch 4 - Model eveything as variable

In [6]:
import cvxpy as cp
import numpy as np

# ---- data ----
n = 6
P_L  = 0.00014218052336201800  # mol/L
kcat = 0.071                   # 1/s
K_exp = 3.7e-6                 # mol/L (single reference value)
S_mu = np.array([4.46510054478746E-07, 2.70874229114477E-04, 6.26189829537269E-04,
                 6.15128391598384E-04, 8.64621184397196E-06, 6.83141171082368E-04])
S_sd = np.array([6.48605511482799E-07, 1.73483135104930E-04, 1.47283282331600E-04,
                 2.64977781270853E-04, 2.04927424163342E-05, 1.30851850873823E-04])
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275]) * 60.0  # s
k_obs = np.log(2.0) / t_half   # target 1/s

# Set S lower bounds strictly positive
eps = 1e-12
S_hi = S_mu + S_sd
S_lo = np.where(S_mu - S_sd > 0.0, S_mu - S_sd, 0.01*S_mu + eps)

# ---- variables ----
K_M = cp.Variable(n, pos=True)     # K_M per substrate
S = cp.Variable(n, pos=True)     # substrate concentrations
k_active = cp.Variable(n, pos=True)     # active degradation rates
tK_M = cp.Variable(n, pos=True)    # symmetric rel. error for K
tk_active = cp.Variable(n, pos=True)    # symmetric rel. error for k

# ---- expressions ----
beta = 1 + cp.sum(cp.multiply(S, cp.inv_pos(K_M)))  # posynomial

# ---- constraints ----
constr = []

# 1. set constraint on possible substrate concentration: S within [S_lo, S_hi]
constr += [ S <= S_hi ]
constr += [ S >= S_lo ]

# Kinetics upper bound: k_i * K_i * beta <= P_L * kcat  (vectorized)
constr += [ cp.multiply(k_active * K_M, beta) <= P_L * kcat ]

# Symmetric relative error for k_cat:  max(k/k_obs, k_obs/k) <= tk
constr += [ cp.multiply(k_active / k_obs, cp.inv_pos(tk_active)) <= 1 ]          # (k/k_obs)/tk <= 1
constr += [ cp.multiply(k_obs * cp.inv_pos(k_active), cp.inv_pos(tk_active)) <= 1 ]  # (k_obs/k)/tk <= 1

# Symmetric relative error for K_M: max(K/K_exp, K_exp/K) <= tK
constr += [ cp.multiply((K_M / K_exp), cp.inv_pos(tK_M)) <= 1 ]
constr += [ cp.multiply((K_exp * cp.inv_pos(K_M)), cp.inv_pos(tK_M)) <= 1 ]

# ---- objective ----
# Put more weight on matching k (half-lives), moderate on K, optional small penalty on beta
w_k, w_K, w_beta = 100.0, 1.0, 0.1
obj = w_k * cp.sum(tk_active) + w_K * cp.sum(tK_M) + w_beta * beta

prob = cp.Problem(cp.Minimize(obj), constr)
prob.solve(gp=True, solver=cp.MOSEK)  # set a GP-capable solver via solver=... if you like (e.g., MOSEK/SCS/ECOS)

# Results:
# K.value, S.value, k.value are the estimates
# tk.value and tK.value ~ 1 means near-perfect fits (symmetric relative error)


/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)
/Users/HeenaSaqib/dev/vEcoli/.venv/lib/python3.12/site-packages/cvxpy/expressions/expression.py:674: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 2 times so far.

  warnings.w

np.float64(1496.1161323040103)

In [48]:
t = S >= S_lo
t.is_dgp()

True

In [49]:
k_active.value

array([3.57514303e-04, 1.32220668e-04, 9.80043918e-05, 5.61560049e-05,
       6.03872181e-05, 7.70583384e-05])

In [50]:
k_obs

array([1.45954602e-04, 5.39788613e-05, 4.00101251e-05, 2.29255928e-05,
       2.46529783e-05, 3.14589346e-05])

In [51]:
K_M.value

array([3.70000009e-06, 6.00305992e-06, 1.33116130e-05, 1.13823494e-05,
       3.70000011e-06, 1.42951431e-05])

In [15]:
#PD03938, G6890-MONOMER, G6737-MONOMER, RPOD-MONOMER, PD02936, RED-THIOREDOXIN2-MONOMER

Inequality(Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,)))

# Approach 6 -

In [457]:
import cvxpy as cp
import numpy as np

# ---- data ----
n = 6
P_L  = 0.00014218052336201800  # mol/L
kcat = 0.071                   # 1/s
# K_exp = 3.7e-6                 # mol/L (single reference value)
K_exp = K_m_solution
S_mu = np.array([4.46510054478746E-07, 2.70874229114477E-04, 6.26189829537269E-04,
                 6.15128391598384E-04, 8.64621184397196E-06, 6.83141171082368E-04])
S_sd = np.array([6.48605511482799E-07, 1.73483135104930E-04, 1.47283282331600E-04,
                 2.64977781270853E-04, 2.04927424163342E-05, 1.30851850873823E-04])
t_half = np.array([79.1510021, 214.018094, 288.738238, 503.910765, 468.6027322, 367.223275]) * 60.0  # s
k_obs = np.log(2.0) / t_half   # target 1/s

# Set S lower bounds strictly positive
eps = 1e-12
S_hi = S_mu + S_sd
S_lo = np.where(S_mu - S_sd > 0.0, S_mu - S_sd, 0.01*S_mu + eps)

# ---- variables ----
K_M = cp.Variable(n, pos=True)     # K_M per substrate
# kcat = cp.Variable(n, pos=True)     # K_M per substrate
S = cp.Variable(n, pos=True)     # substrate concentrations
tK_M = cp.Variable(n, pos=True)    # symmetric rel. error for K
t_S = cp.Variable(n, pos=True)    # symmetric rel. error for k
k_active = cp.Variable(n, pos=True)
lambda_temp = cp.Variable(1, pos=True)

# ---- expressions ----
beta = 1 + cp.sum(cp.multiply(S, cp.inv_pos(K_M)))  # posynomial
# k_active = cp.multiply(cp.inv_pos(beta * K_M), P_L * kcat)

# ---- constraints ----
constr = []

# 1. set constraint on possible substrate concentration: S within [S_lo, S_hi]
constr += [ S <= S_hi ]
constr += [ S >= S_lo ]

# 2. P_L * k_cat/(K_M*beta) <= k_active <= ln(2)/tau_half
# constr += [ cp.multiply(k_active, t_half) <= np.log(2)] #k_active <= ln(2)/tau_half
# constr += [ cp.inv_pos(k_active) * np.log(2)/t_half <= 1 ] #k_active >= ln(2)/tau_half

# 2. set constraint on k_active to be less than P_L * kcat/(K_M*beta)
# constr += [cp.multiply(k_active, K_M) * beta <= P_L * kcat]
constr += [k_active <= P_L * kcat/(K_M*beta)]

# 3. set constraint on k_active to be less than ln(2)/t_half
constr += [k_active >= np.log(2)/t_half]
constr += [(np.log(2)/t_half)/(P_L * kcat/(K_M*beta)) <= tK_M]

constr = []
constr += [ S <= S_hi ]
constr += [ S >= S_lo ]

k_active = P_L * kcat/(K_M*beta)
constr += [ np.log(2)/t_half <= k_active]
lambda_temp

# Set up objective
objective = cp.Minimize(cp.max(kcat))
prob = cp.Problem(objective, constr)
prob.solve(gp=True, solver=cp.MOSEK)

0.071

In [590]:
u = cp.Variable(n, pos=True)
v = cp.Variable()
1 + cp.sum(cp.multiply(S, u))
(1 + S_mu @ u)
cp.log(1 + S @ u)

Expression(UNKNOWN, UNKNOWN, ())

In [593]:
cp.log1p(cp.xexp(u))

Expression(UNKNOWN, NONNEGATIVE, (6,))

In [482]:
cp.inv_pos(1 + cp.sum(cp.multiply(S, u)))

Expression(LOG-LOG CONCAVE, NONNEGATIVE, ())

In [430]:
(P_L * kcat/(K_M*beta)) * tK_M[0]

Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,))

In [484]:
cp.inv_pos(K_M) * cp.inv_pos(1 + cp.sum(cp.multiply(S, u)))

Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,))

In [401]:
(np.log(2)/t_half >= k_active).is_dgp()
cp.min(cp.inv_pos(K_M*beta))

Expression(LOG-LOG CONCAVE, NONNEGATIVE, ())

In [510]:
# 2.1 log transform fu
cp.log(u)-cp.log(1+ S_mu @ u)

Expression(UNKNOWN, UNKNOWN, (6,))

In [536]:
# 3. x = s/km
x = cp.Variable(n, pos=True)

constr = []
constr += [P_L * k_cat/(cp.multiply(K_M, x)) >= np.log(2)/t_half]
constr += [cp.multiply(x, K_M) <= S_mu + S_sd]
constr += [cp.multiply(x, K_M) >= S_lo]
objective = cp.sum(P_L * k_cat/(cp.multiply(K_M, x))) #log-log convex
problem = cp.Problem(cp.Minimize(objective), constr)
problem.solve(gp=True, solver=cp.MOSEK)

np.float64(9.624116110383046)

In [571]:
P_L * k_cat * u * cp.inv_pos(1+ S @ u)

Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,))

In [574]:
v = cp.Variable(6, pos=True)
1 + S @ cp.exp(v)

Expression(LOG-LOG CONVEX, NONNEGATIVE, ())

In [583]:
cp.log1p(S_mu @ cp.exp(v))

Expression(QUASICONVEX, NONNEGATIVE, ())

In [582]:
S_mu @ cp.exp(v)

Expression(CONVEX, NONNEGATIVE, ())

In [451]:
K_M.value

array([8.13379278e-14, 2.19940791e-13, 2.96728727e-13, 5.17855899e-13,
       4.81564865e-13, 3.77385745e-13])

In [438]:
np.log(2)/t_half

array([1.45954602e-04, 5.39788613e-05, 4.00101251e-05, 2.29255928e-05,
       2.46529783e-05, 3.14589346e-05])

In [423]:
P_L * kcat/(K_M.value*beta.value)

array([0.17010681, 0.00241084, 0.00137001, 0.00097783, 0.01250878,
       0.00113223])

In [305]:
beta.value

np.float64(26177817.228517562)

In [263]:
tK_M.value

array([1., 1., 1., 1., 1., 1.])

In [270]:
(np.log(2)/t_half)/(P_L * kcat)

array([14.45837004,  5.34718563,  3.96343237,  2.27102606,  2.44214214,
        3.11634517])

In [269]:
(K_M.value*beta.value)

array([0.06238698, 0.1870101 , 0.25230242, 0.44032102, 0.40312904,
       0.32088337])

In [124]:
cp.multiply(k_active, t_half) <= np.log(2)

Inequality(Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,)))

In [69]:
cp.inv_pos(k_active) <= K_M * beta/(P_L * kcat)

Inequality(Expression(CONVEX, NONNEGATIVE, (6,)))

In [78]:
(P_L * kcat)/(beta * K_M)

Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,))

In [94]:
k_active

Expression(LOG-LOG CONCAVE, NONNEGATIVE, (6,))

In [100]:
k_active >= np.log(2)/t_half

Inequality(Constant(CONSTANT, NONNEGATIVE, (6,)))

In [102]:
cp.inv_pos(k_active) * np.log(2)/t_half <= 1

Inequality(Expression(LOG-LOG CONVEX, NONNEGATIVE, (6,)))

# Approach 7 - Just solve it, disregard optimization

In [12]:
from scipy.optimize import fsolve
kcat = 0.071
K_exp = 3.7e-6       # mol/L (single reference value)
S_mu = np.array([4.46510054478746E-07, 2.70874229114477E-04, 6.26189829537269E-04,
                 6.15128391598384E-04, 8.64621184397196E-06, 6.83141171082368E-04])
def equations(K_m_var):
    beta = np.sum(S_mu/K_m_var) + 1
    eqs = []
    for i in range(6):
        k_active = P_L * kcat / (K_m_var[i] * beta)
        eqs.append(k_active - np.log(2)/t_half[i])
    return eqs

# provide initial guess
K_m_initial_guess = K_exp*np.ones(6)

# solve
K_m_solution = fsolve(equations, K_m_initial_guess)

# print solution
print("Solved K_m values:", K_m_solution)

Solved K_m values: [0.06864628 0.18561416 0.25041764 0.43703302 0.40641098 0.31848635]


In [11]:
# print solution
beta_sol = np.sum(S_mu/K_m_solution) + 1
k_active_sol = P_L * kcat / (K_m_solution * beta_sol)

print("Solved K_m values:", K_m_solution)
print(f'Solved beta: {beta_sol}')
print(f'Solved k_active: {k_active_sol}')

Solved K_m values: [0.06864648 0.1856147  0.25041837 0.4370343  0.40641217 0.31848727]
Solved beta: 1.0075401516086824
Solved k_active: [1.45954602e-04 5.39788613e-05 4.00101251e-05 2.29255928e-05
 2.46529783e-05 3.14589346e-05]


In [10]:
np.log(2)/t_half

array([1.45954602e-04, 5.39788613e-05, 4.00101251e-05, 2.29255928e-05,
       2.46529783e-05, 3.14589346e-05])

In [18]:
# not considering beta
K_m_solution = []
for i in range(6):
    K_m_i = t_half[i]*P_L * kcat / np.log(2) - S_mu[i]
    K_m_solution.append(K_m_i)

In [19]:
K_m_solution

[np.float64(0.06916344301120743),
 np.float64(0.18674339242393964),
 np.float64(0.25168037371861307),
 np.float64(0.4397144737169715),
 np.float64(0.40946792969860457),
 np.float64(0.3202055752383714)]

In [20]:
S_mu

array([6.46510054e-07, 2.70874229e-04, 6.26189830e-04, 6.15128392e-04,
       8.64621184e-06, 6.83141171e-04])

In [ ]:
beta <= f